In [9]:
import requests
import json
import re
import sys
from bs4 import BeautifulSoup

soup = BeautifulSoup('html.parser')
soup

<html><body><p>html.parser</p></body></html>

In [2]:
# 인천광역시 10개 구/군 법정동 코드 (앞 5자리)
INCHEON_DISTRICTS = {"강화군": "28710",
     "계양구": "28245",
     "남동구": "28200",
     "동구": "28140",
     "미추홀구": "28177",  # (구)남구
    "부평구": "28237",
    "서구": "28260",
     "연수구": "28185",
    "옹진군": "28720",
     "중구": "28110"
 }

In [3]:

import xml.etree.ElementTree as ET
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE_URL = "http://apis.data.go.kr/1613000/RTMSDataSvcAptTradeDev/getRTMSDataSvcAptTradeDev"
SERVICE_KEY = "YPxXvm/3jO5hggkbdJFWnhh8IC3VdMlkRvZHh1pkG8eZxXw12ymm4OAW10urfUVN++TzkurcrZ5Os3Gr9P8Kwg=="


# 인천 구별 데이터 저장할 딕셔너리
incheon_real_estate_data = []

# 크롤링할 기간 설정
start_year = 2020
start_month = 1
end_year = 2025
end_month = 2


current_year = start_year
current_month = start_month

# ⏳ 월별 반복문 실행
while (current_year < end_year) or (current_year == end_year and current_month <= end_month):
    deal_ymd = f"{current_year}{current_month:02}"
    print(f"\n⏳ {deal_ymd} 데이터 수집 시작...")

    for gu, code in INCHEON_DISTRICTS.items():
        print(f"📌 {gu} ({code}) 데이터 요청 중...")

        params = {
            "LAWD_CD": code,
            "DEAL_YMD": deal_ymd,
            "serviceKey": SERVICE_KEY,
            "pageNo": 1,
            "numOfRows": 999
        }

        response = requests.get(BASE_URL, params=params)
        response.encoding = "utf-8"

        if response.status_code == 200:
            root = ET.fromstring(response.text)

            items = root.findall(".//item")
            if not items:
                print(f"⚠️ {gu}에 거래 데이터가 없습니다.")
                continue

            # XML 데이터 -> 리스트 변환
            for item in items:
                data = {
                    'aptDong': item.find('aptDong').text if item.find('aptDong') is not None else '',
                    'aptNm': item.find('aptNm').text if item.find('aptNm') is not None else '',
                    'aptSeq': item.find('aptSeq').text if item.find('aptSeq') is not None else '',
                    'bonbun': item.find('bonbun').text if item.find('bonbun') is not None else '',
                    'bubun': item.find('bubun').text if item.find('bubun') is not None else '',
                    'buildYear': item.find('buildYear').text if item.find('buildYear') is not None else '',
                    'buyerGbn': item.find('buyerGbn').text if item.find('buyerGbn') is not None else '',
                    'cdealDay': item.find('cdealDay').text if item.find('cdealDay') is not None else '',
                    'cdealType': item.find('cdealType').text if item.find('cdealType') is not None else '',
                    'dealAmount': item.find('dealAmount').text if item.find('dealAmount') is not None else '',
                    'dealDay': item.find('dealDay').text if item.find('dealDay') is not None else '',
                    'dealMonth': item.find('dealMonth').text if item.find('dealMonth') is not None else '',
                    'dealYear': item.find('dealYear').text if item.find('dealYear') is not None else '',
                    'dealingGbn': item.find('dealingGbn').text if item.find('dealingGbn') is not None else '',
                    'estateAgentSggNm': item.find('estateAgentSggNm').text if item.find('estateAgentSggNm') is not None else '',
                    'excluUseAr': item.find('excluUseAr').text if item.find('excluUseAr') is not None else '',
                    'floor': item.find('floor').text if item.find('floor') is not None else '',
                    'jibun': item.find('jibun').text if item.find('jibun') is not None else '',
                    'landCd': item.find('landCd').text if item.find('landCd') is not None else '',
                    'landLeaseholdGbn': item.find('landLeaseholdGbn').text if item.find('landLeaseholdGbn') is not None else '',
                    'rgstDate': item.find('rgstDate').text if item.find('rgstDate') is not None else '',
                    'roadNm': item.find('roadNm').text if item.find('roadNm') is not None else '',
                    'roadNmBonbun': item.find('roadNmBonbun').text if item.find('roadNmBonbun') is not None else '',
                    'roadNmBubun': item.find('roadNmBubun').text if item.find('roadNmBubun') is not None else '',
                    'roadNmCd': item.find('roadNmCd').text if item.find('roadNmCd') is not None else '',
                    'roadNmSeq': item.find('roadNmSeq').text if item.find('roadNmSeq') is not None else '',
                    'roadNmSggCd': item.find('roadNmSggCd').text if item.find('roadNmSggCd') is not None else '',
                    'roadNmbCd': item.find('roadNmbCd').text if item.find('roadNmbCd') is not None else '',
                    'sggCd': item.find('sggCd').text if item.find('sggCd') is not None else '',
                    'slerGbn': item.find('slerGbn').text if item.find('slerGbn') is not None else '',
                    'umdCd': item.find('umdCd').text if item.find('umdCd') is not None else '',
                    'umdNm': item.find('umdNm').text if item.find('umdNm') is not None else ''
                }
                incheon_real_estate_data.append(data)

        else:
            print(f"❌ {gu} 응답 오류 ({response.status_code})")

    # ✅ 다음 달로 이동
    if current_month == 12:
        current_month = 1
        current_year += 1
    else:
        current_month += 1

# 📦 모든 데이터 DataFrame 변환
df_incheon_real_estate_data = pd.DataFrame(incheon_real_estate_data)

# ✅ 최종 데이터 확인
print("\n📊 최종 데이터 개수:", len(df_incheon_real_estate_data))
print(df_incheon_real_estate_data.head())


⏳ 202001 데이터 수집 시작...
📌 강화군 (28710) 데이터 요청 중...
📌 계양구 (28245) 데이터 요청 중...
📌 남동구 (28200) 데이터 요청 중...
📌 동구 (28140) 데이터 요청 중...
📌 미추홀구 (28177) 데이터 요청 중...
📌 부평구 (28237) 데이터 요청 중...
📌 서구 (28260) 데이터 요청 중...
📌 연수구 (28185) 데이터 요청 중...
📌 옹진군 (28720) 데이터 요청 중...
⚠️ 옹진군에 거래 데이터가 없습니다.
📌 중구 (28110) 데이터 요청 중...

⏳ 202002 데이터 수집 시작...
📌 강화군 (28710) 데이터 요청 중...
📌 계양구 (28245) 데이터 요청 중...
📌 남동구 (28200) 데이터 요청 중...
📌 동구 (28140) 데이터 요청 중...
📌 미추홀구 (28177) 데이터 요청 중...
📌 부평구 (28237) 데이터 요청 중...
📌 서구 (28260) 데이터 요청 중...
📌 연수구 (28185) 데이터 요청 중...
📌 옹진군 (28720) 데이터 요청 중...
⚠️ 옹진군에 거래 데이터가 없습니다.
📌 중구 (28110) 데이터 요청 중...

⏳ 202003 데이터 수집 시작...
📌 강화군 (28710) 데이터 요청 중...
📌 계양구 (28245) 데이터 요청 중...
📌 남동구 (28200) 데이터 요청 중...
📌 동구 (28140) 데이터 요청 중...
📌 미추홀구 (28177) 데이터 요청 중...
📌 부평구 (28237) 데이터 요청 중...
📌 서구 (28260) 데이터 요청 중...
📌 연수구 (28185) 데이터 요청 중...
📌 옹진군 (28720) 데이터 요청 중...
⚠️ 옹진군에 거래 데이터가 없습니다.
📌 중구 (28110) 데이터 요청 중...

⏳ 202004 데이터 수집 시작...
📌 강화군 (28710) 데이터 요청 중...
📌 계양구 (28245) 데이터 요청 중...
📌 남동구 (28200) 데이

In [4]:
real_estate_schema = {
    "sggCd": "법정동시군구코드",
    "umdCd": "법정동읍면동코드",
    "landCd": "법정동지번코드",
    "bonbun": "법정동본번코드",
    "bubun": "법정동부번코드",
    "roadNm": "도로명",
    "roadNmSggCd": "도로명시군구코드",
    "roadNmCd": "도로명코드",
    "roadNmSeq": "도로명일련번호코드",
    "roadNmbCd": "도로명지상지하코드",
    "roadNmBonbun": "도로명건물본번호코드",
    "roadNmBubun": "도로명건물부번호코드",
    "umdNm": "법정동",
    "aptNm": "아파트명",
    "jibun": "지번",
    "excluUseAr": "전용면적",
    "dealYear": "계약년도",
    "dealMonth": "계약월",
    "dealDay": "계약일",
    "dealAmount": "거래금액(만원)",
    "floor": "층",
    "buildYear": "건축년도",
    "aptSeq": "단지 일련번호",
    "cdealType": "해제여부",
    "cdealDay": "해제사유발생일",
    "dealingGbn": "거래유형(중개 및 직거래 여부)",
    "estateAgentSggNm": "중개사소재지(시군구 단위)",
    "rgstDate": "등기일자",
    "aptDong": "아파트 동명",
    "slerGbn": "거래주체정보_매도자(개인/법인/공공기관/기타)",
    "buyerGbn": "거래주체정보_매수자(개인/법인/공공기관/기타)",
    "landLeaseholdGbn": "토지임대부 아파트 여부"
}


In [5]:
df_incheon_real_estate_data.rename(columns=real_estate_schema, inplace=True)
df_incheon_real_estate_data.head()

,아파트 동명,아파트명,단지 일련번호,법정동본번코드,법정동부번코드,건축년도,거래주체정보_매수자(개인/법인/공공기관/기타),해제사유발생일,해제여부,거래금액(만원),...,도로명건물본번호코드,도로명건물부번호코드,도로명코드,도로명일련번호코드,도로명시군구코드,도로명지상지하코드,법정동시군구코드,거래주체정보_매도자(개인/법인/공공기관/기타),법정동읍면동코드,법정동
0,,용진3,28710-5,0394,0001,1991,,,,"8,400",...,00009,00001,4271060,00,28710,0,28710,,25025,강화읍 갑곳리
1,,현대,28710-8,0667,0000,1994,,,,"16,500",...,00191,00014,2157001,01,28710,0,28710,,25025,강화읍 갑곳리
2,,세광엔리치빌1차,28710-16,0663,0000,2001,,,,"15,950",...,00274,00000,3150025,02,28710,0,28710,,31025,선원면 창리
3,,그랑프리타워,28710-389,0175,0001,2015,,,,"8,950",...,00007,00000,4271066,00,28710,0,28710,,25022,강화읍 관청리
4,,까치마을,28710-15,0910,0004,2003,,,,"6,000",...,00023,00002,4271311,01,28710,0,28710,,25027,강화읍 옥림리


In [6]:
df_incheon_real_estate_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163128 entries, 0 to 163127
Data columns (total 32 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   아파트 동명                     163128 non-null  object
 1   아파트명                       163128 non-null  object
 2   단지 일련번호                    163128 non-null  object
 3   법정동본번코드                    163128 non-null  object
 4   법정동부번코드                    163128 non-null  object
 5   건축년도                       163128 non-null  object
 6   거래주체정보_매수자(개인/법인/공공기관/기타)  163128 non-null  object
 7   해제사유발생일                    163128 non-null  object
 8   해제여부                       163128 non-null  object
 9   거래금액(만원)                   163128 non-null  object
 10  계약일                        163128 non-null  object
 11  계약월                        163128 non-null  object
 12  계약년도                       163128 non-null  object
 13  거래유형(중개 및 직거래 여부)          163128 non-null  

In [7]:
# 칼럼 데이터타입 변환환
df_incheon_real_estate_data['거래금액(만원)'] = df_incheon_real_estate_data['거래금액(만원)'].str.replace(",", "").astype(int)
df_incheon_real_estate_data['계약일'] = df_incheon_real_estate_data['계약일'].str.replace(",", "").astype(int)
df_incheon_real_estate_data['계약월'] = df_incheon_real_estate_data['계약월'].str.replace(",", "").astype(int)
df_incheon_real_estate_data['계약년도'] = df_incheon_real_estate_data['계약년도'].str.replace(",", "").astype(int)

In [8]:
import numpy as np

# 전용면적 칼럼을 숫자형으로 변환
df_incheon_real_estate_data['전용면적'] = pd.to_numeric(df_incheon_real_estate_data['전용면적'], errors='coerce')

# 소수점 이하 버리기 (truncate 방식)
df_incheon_real_estate_data['전용면적'] = df_incheon_real_estate_data['전용면적'].apply(np.trunc).astype(int)
df_incheon_real_estate_data['전용면적']

0         82
1         84
2         84
3         30
4         62
          ..
163123    59
163124    84
163125    84
163126    40
163127    84
Name: 전용면적, Length: 163128, dtype: int64

In [9]:
# 평당가격 칼럼 추가
df_incheon_real_estate_data['평당가격'] = round(df_incheon_real_estate_data['거래금액(만원)'] / df_incheon_real_estate_data['전용면적'] , 2)
df_incheon_real_estate_data

,아파트 동명,아파트명,단지 일련번호,법정동본번코드,법정동부번코드,건축년도,거래주체정보_매수자(개인/법인/공공기관/기타),해제사유발생일,해제여부,거래금액(만원),...,도로명건물부번호코드,도로명코드,도로명일련번호코드,도로명시군구코드,도로명지상지하코드,법정동시군구코드,거래주체정보_매도자(개인/법인/공공기관/기타),법정동읍면동코드,법정동,평당가격
0,,용진3,28710-5,0394,0001,1991,,,,8400,...,00001,4271060,00,28710,0,28710,,25025,강화읍 갑곳리,102.44
1,,현대,28710-8,0667,0000,1994,,,,16500,...,00014,2157001,01,28710,0,28710,,25025,강화읍 갑곳리,196.43
2,,세광엔리치빌1차,28710-16,0663,0000,2001,,,,15950,...,00000,3150025,02,28710,0,28710,,31025,선원면 창리,189.88
3,,그랑프리타워,28710-389,0175,0001,2015,,,,8950,...,00000,4271066,00,28710,0,28710,,25022,강화읍 관청리,298.33
4,,까치마을,28710-15,0910,0004,2003,,,,6000,...,00002,4271311,01,28710,0,28710,,25027,강화읍 옥림리,96.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163123,,하늘도시우미린1단지,28110-833,1881,0001,2012,개인,,,27000,...,00000,3149078,01,28110,0,28110,개인,14500,중산동,457.63
163124,,운서SKVIEWskycityⅡ,28110-937,1598,0001,2022,개인,,,48800,...,00000,3149060,00,28110,,28110,개인,14600,운남동,580.95
163125,,e편한세상영종국제도시오션하임,28110-898,1887,0004,2018,개인,,,47500,...,00000,3149082,00,28110,0,28110,개인,14500,중산동,565.48
163126,15,라이프비취2차,28110-28,0027,0107,1981,개인,,,8000,...,00000,4247395,01,28110,0,28110,개인,11800,항동7가,200.00


In [10]:
# 평당가격을 평균 값으로 그룹화
grouped_incheon = df_incheon_real_estate_data.groupby(["법정동", "계약년도", "계약월"], as_index=False).agg({"평당가격": "mean"})
grouped_incheon

,법정동,계약년도,계약월,평당가격
0,가정동,2020,1,394.896200
1,가정동,2020,2,450.272586
2,가정동,2020,3,327.976406
3,가정동,2020,4,255.996857
4,가정동,2020,5,335.192093
...,...,...,...,...
4693,효성동,2024,10,436.586596
4694,효성동,2024,11,381.005152
4695,효성동,2024,12,432.041667
4696,효성동,2025,1,378.685909


In [11]:
# 지역=인천 칼럼 추가
grouped_incheon['지역'] = "인천"
grouped_incheon

,법정동,계약년도,계약월,평당가격,지역
0,가정동,2020,1,394.896200,인천
1,가정동,2020,2,450.272586,인천
2,가정동,2020,3,327.976406,인천
3,가정동,2020,4,255.996857,인천
4,가정동,2020,5,335.192093,인천
...,...,...,...,...,...
4693,효성동,2024,10,436.586596,인천
4694,효성동,2024,11,381.005152,인천
4695,효성동,2024,12,432.041667,인천
4696,효성동,2025,1,378.685909,인천


In [12]:
# 데이터 저장
grouped_incheon.to_csv("incheon.csv", index=False)